<a href="https://colab.research.google.com/github/mnocerino23/Learning-NLP/blob/main/Spam_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now, we are onto building our actual spam classifier 

In [34]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

For this course, we make the decision to vectorize using tf-idf vectorizer

In [35]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

#concat the two engineered features with our vectorized tf-idf term document matrix
#note: the label is not included!
X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head(5)

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
#print all methods
print(dir(RandomForestClassifier))
#print default
print(RandomForestClassifier())

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_feature_names', '_check_n_features', '_compute_oob_predictions', '_estimator_type', '_get_oob_predictions', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score_and_attributes', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'n_features_', 'predict', 'predict_log_proba', 'predict_proba', 'score',

In [38]:
#a few importance attributes! 
#feature_importances - lets you print the features important to your model
#fit - helps you fit the model
#predict - use to predict with test set

#important hyperparameters
#max_depth - how deep each decision tree
#n_estimators - how many decision trees within your random forest

# Explore random forest through cross validation

In [39]:
import sklearn
from sklearn.model_selection import cross_val_score

In [40]:
#setting n_jobs to -1 allows trees to be built quickly in parallel
rf = RandomForestClassifier(n_jobs= -1)

In [41]:
cross_val_score(rf, X_features, data['label'], cv = 5, scoring = "accuracy", n_jobs = -1)

array([0.97755835, 0.97666068, 0.97394429, 0.96585804, 0.97304582])

Explore RandomForestClassifier through test Set

In [42]:
from sklearn.metrics import precision_recall_fscore_support as score 
from sklearn.model_selection import train_test_split

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size = 0.2, random_state= 5)

In [44]:
my_forest = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = my_forest.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [45]:
#body length is the most important feature here!
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse = True)[0:10]

[(0.05773329175959854, 'body_len'),
 (0.04605722677075372, 7350),
 (0.041945567789539485, 1803),
 (0.030106920927657196, 3134),
 (0.02620982614540922, 7027),
 (0.023963505381489506, 4796),
 (0.022604984209551652, 6285),
 (0.02129792168586283, 2031),
 (0.0186276873324068, 6746),
 (0.01773386445889405, 7461)]

In [46]:
y_pred = rf_model.predict(X_test)
#pos_label is the thing we want to predict
precision, recall, fscore, support = score(y_test, y_pred, pos_label= 'spam', average = 'binary')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [47]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision,3), 
                                                         round(recall,3),
                                                         round((y_pred == y_test).sum()/len(y_pred),3)))

Precision: 1.0 / Recall: 0.589 / Accuracy: 0.944


*   100% precision - all things identified as spam were spam 100% of the time
*   55.2% recall - 55% of all spam that has come into your inbox was placed in the spam folder. the other 45% is in your inbox. Not good!

*   Spam/ham predictions are generally 93% accurate

# Our spam filter is not aggresive enough! It's not getting all spam







# Next let's tune our model with GridSearch

In [48]:
def train_forest(n_est, depth):
  rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs = -1)
  rf_model = rf.fit(X_train, y_train)
  y_pred = rf_model.predict(X_test)
  precision, recall, f_score, support = score(y_test, y_pred, pos_label='spam', average = 'binary')
  print('Est: {} / Depth: {} ----- Precision: {} / Recall: {} Accuracy: {}'.format(n_est, depth,
                                        round(precision, 3), round(recall, 3), 
                                        round((y_pred == y_test).sum()/len(y_pred),3)))

# Certain parameters effect the aggresiveness of the model! As the depth increases here, the recall score is higher. Estimators isn't affecting it as much. So we want high depth

In [49]:
#implementing grid search ourselves
for n_est in [10,50,100]:
  for depth in [10,20,30,None]:
    train_forest(n_est, depth)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Est: 10 / Depth: 10 ----- Precision: 1.0 / Recall: 0.272 Accuracy: 0.901


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Est: 10 / Depth: 20 ----- Precision: 1.0 / Recall: 0.556 Accuracy: 0.94


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


...

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Est: 100 / Depth: 30 ----- Precision: 1.0 / Recall: 0.642 Accuracy: 0.952


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Est: 100 / Depth: None ----- Precision: 0.991 / Recall: 0.748 Accuracy: 0.965


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


# Now take things a step further with GridSearchCV

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
from sklearn.model_selection import GridSearchCV

In [ ]:
rf = RandomForestClassifier()
param = {
    'n_estimators': [10,15,300],
    'max_depth': [30,60,90,None]
}

gs = GridSearchCV(rf, param, cv = 5, n_jobs = -1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values("mean_test_scores", ascending = False)[0:5]

Having run it for tf-idf, we run again for count vectorization

In [ ]:
rf = RandomForestClassifier()
param = {
    'n_estimators': [10,15,300],
    'max_depth': [30,60,90,None]
}

gs = GridSearchCV(rf, param, cv = 5, n_jobs = -1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values("mean_test_scores", ascending = False)[0:5]

Now, try another classification algorithm: Gradient Boosting
- also built on decision trees but uses boosting whereas random forest uses bagging
- uses a lot of very basic decision trees and learns from their mistakes
- no n-jobs parameter here (you can't parallelize gradient boosting since each iteration builds on the prior iteration)
- however, learning rate is a new hyperparemeter

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#take a look at the attributes and methods
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size = 0.2, random_state= 5)

In [ ]:
def train_GB(est, max_depth, learn_rate):
  gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, 
                                  learning_rate = learn_rate)
  gb_model = gb.fit(X_train, y_train)
  y_pred = gb_model.predict(X_test)
  precision, recall, fscore, support = score(y_test, y_pred, pos_label = 'spam', average = 'binary')
  print('Est: {} / Depth: {} / LR: {} ------ Precision: {}, Recall: {}, Accuracy: {}'.format(
      est, max_depth, learn_rate, round(precision, 3), round(recall, 3), round((y_pred == y_test).sum()/(len(y_pred)), 3)))

In [ ]:
#this takes a super long time to run! since can't run in parallel
for n_est in [50, 100, 150]:
  for max_depth in [3, 7, 11, 15]:
    for lr in [0.01, 0.1, 1]:
      train_GB(n_est, max_depth, lr)

# Analyzing the results:
worst models had 0.01 learning rate and low number of estimators
Best hyperparemeters:
- learning rate of 0.1
- high number of estimators
- high max depth

Now evaluate our gradient boosting model with gridsearchcv

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

In [30]:
from sklearn.model_selection import GridSearchCV

This will take a long time to run! Note: fit times are much larger for gradient boosting

In [33]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100,150], 
    'max_depth': [7,11,15],
    'learning_rate': [0.1]
}

#models and subsets trained in parallel but trees will not be trained in parallel
gs = GridSearchCV(gb, param, cv = 5, n_jobs = -1)
cv_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.results_).sort_values('mean_test_score', ascending = False)[0:5]

KeyboardInterrupt: ignored

In [ ]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100,150], 
    'max_depth': [7,11,15],
    'learning_rate': [0.1]
}

#models and subsets trained in parallel but trees will not be trained in parallel
gs = GridSearchCV(gb, param, cv = 5, n_jobs = -1)
cv_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.results_).sort_values('mean_test_score', ascending = False)[0:5]